In [13]:
from flask import Flask
from flask import render_template
from flask import request

import requests
from bs4 import BeautifulSoup
import re

app = Flask(__name__)

In [2]:
def search_google(keyword, start_page, end_page=None):
    url = "https://www.google.com/search?q={0}+magnet%3A%3Fxt%3D&oq={0}+magnet%3A%3Fxt%3D&start={1}".format(keyword, start_page)
    # Google, Naver와 같은 Search Engine등 모바일과 브라우저의 양식이 다르기 때문에, 브라우저에서 접속한다는 것을 알림
    header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36,gzip(gfe)"}
    r = requests.get(url, headers=header)
    bs = BeautifulSoup(r.text, "lxml")
    
    links = bs.select("div.g > div.tF2Cxc > div.yuRUbf > a")
    results = []

    if end_page is None:
        # 페이징 될 경우 검색결과 약 100,000개 중 2페이지 (0.50초) 이런 형식으로 결과가 출력됩니다.
        # 그래서 "검색결과 약" 과 "개" 사이의 결과 갯수를 파싱하기 위해서 아래처럼 작업합니다.
        parse_text_1 = "검색결과 약"
        parse_text_2 = "개"
        # 최초의 검색 결과를 div 태그의 ID가 result-stats 인 요소의 text 값을 구합니다.
        # 기존의 코드에서 ID 이름이 변경되었습니다.
        text = bs.select("div#result-stats")[0].text

        # "검색결과 약" 에서부터 맨 뒤까지 문자열을 슬라이싱 해서 text에 다시 담습니다.
        text = text[text.find(parse_text_1) + len(parse_text_1):]
        # "검색결과 약" 이 제거된 text 에서 "개" 를 찾아 그곳까지 슬라이싱해서 다시 text에 담습니다.
        text = text[:text.find(parse_text_2)]
        # 최종적으로 , 를 제거하고 공백제거 후 counts 변수에 담으면 숫자만 담겨 집니다.
        counts = text.replace(",", "").strip()

        # 이전코드
        # counts = bs.select("div#result-stats")[0].text.replace("검색결과 약", "").replace("개", "").replace(",", "").split("(")[0].strip()

        #print(counts)
        end_page = int(int(counts) / 10)
        if end_page > 10:
            end_page = 10

    for a in links:
        href = a["href"]
        text = a.select("h3")
        
        if len(text) <= 0:
            continue
        title = text[0].text

        try:
            r = requests.get(href)
            bs = BeautifulSoup(r.text, "lxml")
            #                        compile:  패턴을 컴파일 해서 재활용하는 목적
            #                        r: rawstring으로 받아들여라, \\ --> \           
            magnets = bs.find_all("a", href=re.compile(r'magnet:\?xt=*'))

            if len(magnets) > 0:
                magnet = magnets[0]["href"]
                results.append({
                    'magnet': magnet,
                    'title': title
                })
        except:
            pass
    if start_page < end_page:
        start_page += 10
        results.extend(search_google(keyword, start_page, end_page=end_page))

    return results

In [14]:
@app.route('/', methods=['GET', 'POST'])
def index():
    if 'keyword' in request.form:
        keyword = request.form['keyword']
        results = search_google(keyword, 0)
    # 맨 처음 랜더링된 화면, 검색값이 없을 때, GET
    else:
        results = []
        
    if len(results) > 0:
        return render_template('index.html', **({'magnets': results}))
    else:
        return render_template('index.html')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=9995, debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1